# Imports

In [1]:
import numpy as np
import heapq
import pandas as pd

# Constructing a matrix representing the layout

In [4]:
def divide_aisles(nbMidAisles):
    if nbMidAisles == 0:
        return [(5,29)]
    elif nbMidAisles == 1:
        return [(4,16), (17,29)]
    elif nbMidAisles == 2:
        return [(3,11), (12,20), (21,29)]
    elif nbMidAisles == 3:
        return [(2,8), (9,15), (16,22), (23,29)]
    elif nbMidAisles == 4:
        return [(1,5), (6,11), (12,17), (18,23), (24,29)]

def construct_warehouse_layout(nbMidAisles):
    '''
    Function which takes the number of middle aisles required and 
    returns a warehouse layout with the added aisles in the warehouse.
    '''

    # Define the shape of the grid
    rows = 30
    cols = 13

    filled_squares = [] # initialise shelves to fill

    # define where to place the middle aisles and add shelves accordingly
    row_ranges = divide_aisles(nbMidAisles)
    columns = [1,3,4,6] # columns to add the shelves to
    for start, end in row_ranges:
        for row in range(start, end):
            for col in columns:
                filled_squares.append((row, col))

    # Initialize the grid as zeros
    grid = np.zeros((rows, cols), dtype=int)

    # Mark filled squares as 1
    for row,col in filled_squares:
        grid[row, col] = 1

    # Find indices that define the corridors between columns of shelves
    adjacent_shelves = []
    for i in range(len(grid)):
        # Iterate over each column, skipping the first and last columns
        for j in range(1, len(grid[i]) - 1):
            # Check if current element is '0' and both its neighbors are '1'
            if grid[i][j] == 0 and grid[i][j - 1] == 1 and grid[i][j + 1] == 1:
                adjacent_shelves.append([i, j])  # Append indices to the list
    return grid, adjacent_shelves

# $\text{A}^*$ Algorithm
#### This is a pathfinding algorithm which calculates the Manhattan distance between points while avoiding obstacles (shelves).

In [6]:
class Shelf:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0
        self.h = 0

    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)

def get_neighbours(shelf, warehouse_grid):
    neighbours = []
    moves = [(1, 0), (-1, 0), (0, 1), (0, -1)]
    for dx, dy in moves:
        x, y = shelf.x + dx, shelf.y + dy
        if 0 <= x < len(warehouse_grid) and 0 <= y < len(warehouse_grid[0]) and warehouse_grid[x][y] == 0:
            neighbours.append(Shelf(x, y, shelf))
    return neighbours

def manhattan_distance(shelf1, shelf2):
    return abs(shelf1.x - shelf2.x) + abs(shelf1.y - shelf2.y)

def a_star(start, destination, warehouse_grid):
    open_list = []
    closed_set = set()
    heapq.heappush(open_list, start)

    while open_list:
        current_shelf = heapq.heappop(open_list)

        if (current_shelf.x, current_shelf.y) == (destination.x, destination.y):
            path = []
            while current_shelf:
                path.append((current_shelf.x, current_shelf.y))
                current_shelf = current_shelf.parent
            return path[::-1]

        closed_set.add((current_shelf.x, current_shelf.y))

        for neighbour in get_neighbours(current_shelf, warehouse_grid):
            if (neighbour.x, neighbour.y) in closed_set:
                continue
            neighbour.g = current_shelf.g + 1
            neighbour.h = manhattan_distance(neighbour, destination)
            heapq.heappush(open_list, neighbour)

    return None

def calculate_path_and_distance(start, end, warehouse_grid):
    path = a_star(Shelf(*start), Shelf(*end), warehouse_grid)
    if path:
        total_distance = sum(manhattan_distance(Shelf(*path[i]), Shelf(*path[i+1])) for i in range(len(path)-1))
        return path, total_distance
    else:
        return None, None

# Using $\text{A}^*$ to construct the distance matrix

In [7]:
def construct_distance_matrix(warehouse_layout, shelf_positions):

    shelf_positions = sorted(shelf_positions, key = lambda shelf: (shelf[1], -shelf[0]))

    distance_matrix = np.zeros((len(shelf_positions), len(shelf_positions)), dtype = int)
    paths = {}
    for i, start_point in enumerate(shelf_positions):
        for j, end_point in enumerate(shelf_positions):
            path, total_distance = calculate_path_and_distance(start_point, end_point, warehouse_layout)
            distance_matrix[i,j] = total_distance
            paths[f"{start_point} to {end_point}"] = path
            # if path:
            #     print(f"Path from {start_point} to {end_point}:", path, total_distance)
            #     # print("Total Manhattan Distance:", total_distance)
            # else:
            #     print("No path found.")
            
    quad_quad_mat = np.zeros((96,96), dtype = int)

    TL_quad = distance_matrix[:24, :24]
    TR_quad = distance_matrix[:24, -24:]
    BL_quad = distance_matrix[-24:, :24]
    BR_quad = distance_matrix[-24:, -24:]

    quad_quad_mat[:24, :24] = quad_quad_mat[:24, 24:48] = quad_quad_mat[24:48, :24] = quad_quad_mat[24:48, 24:48] = TL_quad

    quad_quad_mat[:24, 48:72] = TR_quad
    quad_quad_mat[:24, 72:] = TR_quad
    quad_quad_mat[24:48, 48:72] = TR_quad
    quad_quad_mat[24:48, 72:] = TR_quad

    quad_quad_mat[48:72, :24] = BL_quad
    quad_quad_mat[48:72, 24:48] = BL_quad
    quad_quad_mat[72:, :24] = BL_quad
    quad_quad_mat[72:, 24:48] = BL_quad

    quad_quad_mat[48:72, 48:72] = BR_quad
    quad_quad_mat[48:72, 72:] = BR_quad
    quad_quad_mat[72:, 48:72] = BR_quad
    quad_quad_mat[72:, 72:] = BR_quad

    return quad_quad_mat, paths

In [8]:
def construct_door_shelf_distances(warehouse_layout, shelf_positions, door_position = [29, 3]):
    shelf_positions = sorted(shelf_positions, key = lambda shelf: (shelf[1], -shelf[0]))
    init_door_dist = np.zeros(len(shelf_positions) + 1, dtype = int)
    paths = {}

    for i, shelf in enumerate(shelf_positions):
        path, total_distance = calculate_path_and_distance(door_position, shelf, warehouse_layout)
        init_door_dist[i+1] = total_distance
        paths[f"Door to {shelf}"] = path
        # if path:
        #     print(f"Path from door to {shelf}:", path, total_distance)
        # else:
        #     print("No path found.")
    
    first_half = init_door_dist[1:25]
    second_half = init_door_dist[25:]

    door_shelf_dist = np.zeros(len(shelf_positions*2) + 1, dtype = int)
    door_shelf_dist[1:25] = first_half
    door_shelf_dist[25:49] = first_half
    door_shelf_dist[49:73] = second_half
    door_shelf_dist[73:] = second_half

    return door_shelf_dist, paths

# Writing distance matrices (squares and metres) to Excel files

In [25]:
for i in range(5):
    warehouse_layout, adjacent_shelves = construct_warehouse_layout(i)
    distance_mat = construct_distance_matrix(warehouse_layout, adjacent_shelves)[0]
    door_shelf_dist_vec = construct_door_shelf_distances(warehouse_layout, adjacent_shelves)[0]

    full_distance_matrix_squares = np.zeros((len(door_shelf_dist_vec), len(door_shelf_dist_vec)), dtype = int)
    full_distance_matrix_squares[0, 1:] = full_distance_matrix_squares[1:, 0] = door_shelf_dist_vec[1:]
    full_distance_matrix_squares[1:, 1:] = distance_mat

    full_distance_matrix_metres = full_distance_matrix_squares*3

    full_distance_matrix_squares_df = pd.DataFrame(full_distance_matrix_squares)
    full_distance_matrix_metres_df = pd.DataFrame(full_distance_matrix_metres)

    with pd.ExcelWriter(f"Distance Matrices/DistanceMatrix_{i}_aisles.xlsx") as writer:  
        full_distance_matrix_squares_df.to_excel(writer, sheet_name="DistanceMatrixSquares", index = False)
        full_distance_matrix_metres_df.to_excel(writer, sheet_name="DistanceMatrixMetres", index = False)